In [ ]:
#Configuration environment
import os

os.environ['KAGGLE_USERNAME'] = "ahmedgamal12" # username from the json file
os.environ['KAGGLE_KEY'] = "6967fdeab2dcdc7361ec924e3ba0dafb" # key from the json file

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
import zipfile

with zipfile.ZipFile("/content/dogs-vs-cats.zip", mode="r") as archive:
    archive.printdir()
    archive.extractall()

In [ ]:
with zipfile.ZipFile("/content/train.zip", mode="r") as archive:
    archive.extractall()

In [ ]:
with zipfile.ZipFile("/content/test1.zip", mode="r") as archive:
    archive.extractall()

In [ ]:
# import libs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import os
import glob

In [ ]:
# make main folder and move train , test in it
os.mkdir('main')
os.mkdir('main/train')
os.mkdir('main/test')

In [ ]:
len(os.listdir('/content/train')),len(os.listdir('/content/test1'))

In [ ]:
import shutil

In [ ]:
print(os.listdir('/content/train'))

In [ ]:
os.rename('/content/main/test','/content/main/test1')

In [ ]:
arr=['train','test1']

for folder in arr:
  path=os.path.join('/content/',folder)
  for img in os.listdir(path):
    img_path=os.path.join(path,img)
    target=f'/content/main/{folder}'
    shutil.move(img_path, target)

In [ ]:
print(os.listdir('/content/main/test1'))

In [ ]:
print(os.listdir('/content/main/train'))

In [ ]:
os.rmdir('/content/test1')
os.rmdir('/content/train')

In [ ]:
os.rename('/content/main/test1','/content/main/test')

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# image generator libs
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
# make 2 classes in train ==> dog & cat
os.mkdir('/content/main/train/cat')
os.mkdir('/content/main/train/dog')

In [ ]:
len(os.listdir('/content/main/train'))

In [ ]:
# len(glob.glob('/content/main/train/*.jpg'))
for i in glob.glob('/content/main/train/*.jpg'):
  last=i.split('/')[-1]
  if last.split('.')[0]=='cat':
    shutil.move(i, '/content/main/train/cat')
  else:
    shutil.move(i,'/content/main/train/dog')

In [ ]:
len(os.listdir('/content/main/train/cat')),len(os.listdir('/content/main/train/dog'))

In [ ]:
train_batch=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory('/content/main/train',
                    target_size=(224,224),batch_size=10)

In [ ]:
imgs,labels=next(train_batch)

In [ ]:
plt.imshow(imgs[6])

In [ ]:
labels

In [ ]:
vgg_16_model=tf.keras.applications.vgg16.VGG16()
vgg_16_model.summary()

In [ ]:
model=keras.Sequential()
for layer in vgg_16_model.layers[:-1]:
  model.add(layer)

In [ ]:
# make freezing for this model
for layer in model.layers:
  layer.trainable=False

In [ ]:
model.add(tf.keras.layers.Dense(2,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_batch,steps_per_epoch=len(train_batch),epochs=10,verbose=2)

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
import random
x=random.choice(os.listdir('/content/main/test'))
print(x)
img=plt.imread(os.path.join('/content/main/test',x))
img=cv2.resize(img,dsize=(224,224))
img_new=preprocess_input(img)
plt.imshow(img)

In [ ]:
feature=model.predict(np.expand_dims(img_new,0))

In [ ]:
feature.round(2)

In [ ]:
train_batch.class_indices

In [ ]:
train_batch.class_mode

In [ ]:
model.save('/content/drive/MyDrive/cat_dog.h5')

## this is the end of fine tune vgg16
in the down is an Experiments alone

In [ ]:
labels={'cat':0,'dog':1}

def get_label(num):
  for i,j in labels.items():
    if j==num:return i
get_label(0)

In [ ]:
imgs=[]
target=[]
for i in os.listdir('/content/main/train'):
  img=os.path.join('/content/main/train',i)
  img_arr=plt.imread(img)
  new_arr=cv2.resize(img_arr,dsize=(64,64))
  label=i.split('.')[0]
  imgs.append(new_arr)
  target.append(labels[label])

In [ ]:
np.shape(imgs)

In [ ]:
X=np.array(imgs)
y=np.array(target)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test ,y_train,y_test = train_test_split(X, y, test_size=0.20, random_state=88)

In [ ]:
array_imgs=np.array(imgs)

In [ ]:
plt.imshow(array_imgs[0])

In [ ]:
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   width_shift_range = 1,
                                   rotation_range=90,
                                   height_shift_range = 1)

it=data_generator_with_aug.flow(array_imgs,batch_size=25000)

In [ ]:
images=[next(it)[0].astype(np.uint8) for i in range(15000)]

In [ ]:
len(images)

In [ ]:
# mobile=tf.keras.applications.mobilenet.MobileNet()
# mobile.summary()

In [ ]:
# x = mobile.layers[-6].output
# x

In [ ]:
output = Dense(units=2, activation='softmax')(x)


In [ ]:
# model = Model(inputs=mobile.input, outputs=output)

In [ ]:
# for layer in model.layers[:-23]:
#     layer.trainable = False

In [ ]:
# model.summary()

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(3,3), activation='relu', input_shape=(64,64,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=56, kernel_size=(5,5), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [ ]:
model.fit(x=X_train,y=y_train,steps_per_epoch=len(X_train),epochs=30)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.evalute(X_test,y_test)